# Experiment training for model v0.3
- model from `models/model_v0_3.py`

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.losses import BinaryCrossentropy

In [2]:
G_train_df      = pd.read_csv ('../data/preprocessed_data/G_train.csv')
T_train_df      = pd.read_csv ('../data/preprocessed_data/T_train.csv')
G_T_train_df    = pd.read_csv ('../data/preprocessed_data/balanced_G_T_train.csv')

G_cv_df         = pd.read_csv ('../data/preprocessed_data/G_cv.csv')
T_cv_df         = pd.read_csv ('../data/preprocessed_data/T_cv.csv')
G_T_cv_df       = pd.read_csv ('../data/preprocessed_data/G_T_cv.csv')


In [3]:
random_state = 13
frac = 0.1

G_train_df      = G_train_df.sample(frac = frac, random_state=random_state)
T_train_df      = T_train_df.sample(frac = frac, random_state=random_state)
G_T_train_df    = G_T_train_df.sample(frac = frac, random_state=random_state)

In [4]:
y_train = G_T_train_df.drop(columns = ['group ID','technique ID' ]).values
y_train.dtype

# G_train = sampled_G_train_df.drop(columns = ids)
G_train = G_train_df.drop(columns = 'group ID').values

# T_train = sampled_T_train_df.drop(columns = ids)
T_train = T_train_df.drop(columns = 'technique ID').values

In [5]:
G_train_tf = tf.convert_to_tensor(G_train)
T_train_tf = tf.convert_to_tensor(T_train)
y_train_tf = tf.convert_to_tensor(y_train)
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))

batch_size = 32


train_dataset = train_dataset.shuffle(buffer_size=len(G_train_tf))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [6]:
y_cv = G_T_cv_df.drop(columns = ['group ID','technique ID' ]).values
y_cv.dtype

# G_cv = sampled_G_cv_df.drop(columns = ids)
G_cv = G_cv_df.drop(columns = 'group ID').values

# T_cv = sampled_T_cv_df.drop(columns = ids)
T_cv = T_cv_df.drop(columns = 'technique ID').values

In [7]:
G_cv_tf = tf.convert_to_tensor(G_cv)
T_cv_tf = tf.convert_to_tensor(T_cv)
y_cv_tf = tf.convert_to_tensor(y_cv)
cv_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_cv_tf, 'input_Technique': T_cv_tf}, y_cv_tf))
cv_dataset = cv_dataset.batch(batch_size)

In [10]:
import sys
sys.path.append('../models') 

from model_v0_3 import ContentBasedFiltering

In [9]:
# input shapes config
num_G_features = G_train.shape[1]  # remove Group ID during training
num_T_features = T_train.shape[1]   # remove Movie ID during training

model1 = ContentBasedFiltering (
    num_G_features=num_G_features, 
    num_T_features=num_T_features,
    Group_NN_width = 128,
    Group_NN_depth = 2,
    Technique_NN_width = 128,
    Technique_NN_depth = 2,)

In [10]:
model1.model.summary()

Model: "recsysNN_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_Group (InputLayer)       [(None, 463)]        0           []                               
                                                                                                  
 input_Technique (InputLayer)   [(None, 54)]         0           []                               
                                                                                                  
 Group_NN (Sequential)          (None, 32)           80032       ['input_Group[0][0]']            
                                                                                                  
 Technique_NN (Sequential)      (None, 32)           27680       ['input_Technique[0][0]']        
                                                                                     

In [11]:
epochs = 50
history = model1.train (
    train_dataset,
    cv_dataset,
    epochs= epochs
)

Epoch 1/50
403/403 [==============================] - 3s 5ms/step - loss: 0.5640 - val_loss: 0.5406
Epoch 2/50
403/403 [==============================] - 2s 5ms/step - loss: 0.4917 - val_loss: 0.4238
Epoch 3/50
403/403 [==============================] - 2s 5ms/step - loss: 0.4658 - val_loss: 0.5539
Epoch 4/50
403/403 [==============================] - 2s 4ms/step - loss: 0.4478 - val_loss: 0.4029
Epoch 5/50
403/403 [==============================] - 2s 4ms/step - loss: 0.4358 - val_loss: 0.4435
Epoch 6/50
403/403 [==============================] - 2s 4ms/step - loss: 0.4316 - val_loss: 0.4432
Epoch 7/50
403/403 [==============================] - 2s 4ms/step - loss: 0.4328 - val_loss: 0.4890
Epoch 8/50
403/403 [==============================] - 2s 5ms/step - loss: 0.4268 - val_loss: 0.4687
Epoch 9/50
403/403 [==============================] - 2s 5ms/step - loss: 0.4292 - val_loss: 0.5035
Epoch 10/50
403/403 [==============================] - 2s 5ms/step - loss: 0.4124 - val_loss: 0.4995

In [12]:
results = model1.evaluate (train_dataset)

403/403 [==============================] - 1s 2ms/step - loss: 0.4166


In [13]:
model1.predict(train_dataset)

403/403 [==============================] - 1s 2ms/step


array([[-1.2294296 ],
       [-1.7281423 ],
       [ 0.3826456 ],
       ...,
       [ 1.2851872 ],
       [-1.9019394 ],
       [-0.13766348]], dtype=float32)